In [11]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement nvidia-nccl-cu12==2.21.5 (from versions: 0.0.1.dev5)
ERROR: No matching distribution found for nvidia-nccl-cu12==2.21.5


## Load REMI Tokenizer

In [1]:
import pickle

REMI_TOK_PATH = "./vocab_remi.pkl"  # Path to the saved REMI tokenizer

with open(REMI_TOK_PATH, "rb") as f:
    tokenizer = pickle.load(f)

print(f"Loaded tokenizer with vocab size {tokenizer.vocab_size}")

REMI_TOKENIZER = tokenizer

c:\Users\josep\miniforge3\envs\csc413\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded tokenizer with vocab size 524


In [2]:
# Confirm that the tokenizer works
test_file = "./midicaps_data/lmd_full/0/0a0a2b0e4d3b7bf4c5383ba025c4683e.mid"
tokenizer(test_file)

ValueError: midicaps_data\lmd_full\0\0a0a2b0e4d3b7bf4c5383ba025c4683e.mid is not a file

# GigaMIDI

In [29]:
from datasets import load_dataset

gigamidi2 = load_dataset("Metacreation/GigaMIDI", "v2.0.0", split="train")
gigamidi2

Dataset({
    features: ['music', 'NOMML', 'num_tracks', 'TPQN', 'total_notes', 'avg_note_duration', 'avg_velocity', 'min_velocity', 'max_velocity', 'tempo', 'loop_track_idx', 'loop_instrument_type', 'loop_start', 'loop_end', 'loop_duration_beats', 'loop_note_density', 'NOMML_Instrument', 'Type', 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2', 'md5', 'music_styles_curated', 'music_style_scraped', 'music_style_audio_text_Discogs', 'music_style_audio_text_Lastfm', 'music_style_audio_text_Tagtraum', 'title', 'artist', 'audio_text_matches_score', 'audio_text_matches_sid', 'audio_text_matches_mbid', 'MIDI program number (expressive)', 'instrument_group (expressive)', 'start_tick (expressive)', 'end_tick (expressive)', 'duration_beats (expressive)', 'note_density (expressive)', 'loopability (expressive)'],
    num_rows: 1708973
})

### Sample Dataset (Optional)

In [45]:
import random

num_samples = 10

# Generate random indices
random_indices = random.sample(range(len(gigamidi2)), num_samples)

# Select the random samples
sample = gigamidi2.select(random_indices)

print(sample)

Dataset({
    features: ['music', 'NOMML', 'num_tracks', 'TPQN', 'total_notes', 'avg_note_duration', 'avg_velocity', 'min_velocity', 'max_velocity', 'tempo', 'loop_track_idx', 'loop_instrument_type', 'loop_start', 'loop_end', 'loop_duration_beats', 'loop_note_density', 'NOMML_Instrument', 'Type', 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2', 'md5', 'music_styles_curated', 'music_style_scraped', 'music_style_audio_text_Discogs', 'music_style_audio_text_Lastfm', 'music_style_audio_text_Tagtraum', 'title', 'artist', 'audio_text_matches_score', 'audio_text_matches_sid', 'audio_text_matches_mbid', 'MIDI program number (expressive)', 'instrument_group (expressive)', 'start_tick (expressive)', 'end_tick (expressive)', 'duration_beats (expressive)', 'note_density (expressive)', 'loopability (expressive)'],
    num_rows: 10
})


## Filter Dataset

In [40]:
from pathlib import Path
from miditok.constants import SCORE_LOADING_EXCEPTION
from miditok.utils import get_bars_ticks
from symusic import Score

def is_long_enough(
    midi_bytes, min_num_bars, min_num_notes
) -> bool:
    """
    Check if ``midi_bytes`` contains the minimum required number of notes and
    bars.

    :param score: ``symusic.Score`` to inspect or path to a MIDI file.
    :param min_num_bars: minimum number of bars the score should contain.
    :param min_num_notes: minimum number of notes that score should contain.
    :return: boolean indicating if ``score`` is valid.
    """
    # Load score from bytes
    try:
        score = Score.from_midi(midi_bytes)
    except SCORE_LOADING_EXCEPTION:
        return False

    # reject files with no time signatures
    if len(score.time_signatures) == 0:
        return False

    try:
        bars = get_bars_ticks(score)
    except Exception:
        return False

    return len(bars) >= min_num_bars and score.note_num() > min_num_notes

In [43]:
ds = gigamidi2
# ds = sample
print(f"Original dataset size: {len(ds)}")

# Filter out drum-only tracks
filter_category = 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2'
ds = ds.filter(lambda x: x[filter_category] == 0)

print(f"Dataset size after filtering drum-only tracks: {len(ds)}")

# Filter midi files with less than 8 bars and 32 notes
min_num_bars = 8
min_num_notes = 32
ds = ds.filter(
    lambda x: is_long_enough(x["music"], min_num_bars, min_num_notes)
)

print(f"Dataset size after filtering short files: {len(ds)}")

Original dataset size: 1708973
Dataset size after filtering drum-only tracks: 1028729


Filter: 100%|██████████| 1028729/1028729 [03:52<00:00, 4419.80 examples/s]

Dataset size after filtering short files: 420314


In [ ]:
ds

# Dataset({
#     features: ['music', 'NOMML', 'num_tracks', 'TPQN', 'total_notes', 'avg_note_duration', 'avg_velocity', 'min_velocity', 'max_velocity', 'tempo', 'loop_track_idx', 'loop_instrument_type', 'loop_start', 'loop_end', 'loop_duration_beats', 'loop_note_density', 'NOMML_Instrument', 'Type', 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2', 'md5', 'music_styles_curated', 'music_style_scraped', 'music_style_audio_text_Discogs', 'music_style_audio_text_Lastfm', 'music_style_audio_text_Tagtraum', 'title', 'artist', 'audio_text_matches_score', 'audio_text_matches_sid', 'audio_text_matches_mbid', 'MIDI program number (expressive)', 'instrument_group (expressive)', 'start_tick (expressive)', 'end_tick (expressive)', 'duration_beats (expressive)', 'note_density (expressive)', 'loopability (expressive)'],
#     num_rows: 420314
# })

Dataset({
    features: ['music', 'NOMML', 'num_tracks', 'TPQN', 'total_notes', 'avg_note_duration', 'avg_velocity', 'min_velocity', 'max_velocity', 'tempo', 'loop_track_idx', 'loop_instrument_type', 'loop_start', 'loop_end', 'loop_duration_beats', 'loop_note_density', 'NOMML_Instrument', 'Type', 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2', 'md5', 'music_styles_curated', 'music_style_scraped', 'music_style_audio_text_Discogs', 'music_style_audio_text_Lastfm', 'music_style_audio_text_Tagtraum', 'title', 'artist', 'audio_text_matches_score', 'audio_text_matches_sid', 'audio_text_matches_mbid', 'MIDI program number (expressive)', 'instrument_group (expressive)', 'start_tick (expressive)', 'end_tick (expressive)', 'duration_beats (expressive)', 'note_density (expressive)', 'loopability (expressive)'],
    num_rows: 420314
})

In [49]:
isinstance(sample[0]['music'], bytes)

True

## Pretokenize GigaMIDI

In [55]:
import os
import torch
from tqdm import tqdm
from pathlib import Path
from multiprocessing import Pool
import tempfile

def process_item(args):
    (
        item,
        out_folder,
        remi_tokenizer,
        max_decoder_len,
    ) = args

    file_id = item["md5"]                # unique id for output naming
    out_path = os.path.join(out_folder, file_id + ".pt")
    # Skip if already exists
    if os.path.exists(out_path):
        return None

    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # ---- Write temporary MIDI ----
    midi_bytes = item["music"]            # HF stores as bytes/string

    with tempfile.NamedTemporaryFile(suffix=".mid", delete=False) as tmp:
        tmp.write(midi_bytes)
        tmp_path = tmp.name

    # ---- Tokenize MIDI ----
    midi = None
    try:
        tokens = remi_tokenizer(tmp_path)

        if len(tokens.ids) == 0:
            # empty tokenization → treat as skip or BOS/EOS?
            # Here we skip because the file probably failed parsing
            print(f"[WARN] Empty token sequence for item {base}, skipping.")
            return "SKIP"

        midi = (
            [remi_tokenizer["BOS_None"]]
            + tokens.ids
            + [remi_tokenizer["EOS_None"]]
        )

    except Exception as e:
        print(f"[ERROR] Tokenization failed for item {base}: {e}")
        return "SKIP"

    finally:
        # always clean up temp file
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

    # ---- Pad/truncate ----
    if len(midi) < max_decoder_len:
        labels = torch.nn.functional.pad(
            torch.tensor(midi),
            (0, max_decoder_len - len(midi))
        )
    else:
        labels = torch.tensor(midi[:max_decoder_len])

    labels = labels.long()

    # ---- Save ----
    torch.save({"labels": labels}, out_path)

    return None

def pretokenize_gigamidi(
    hf_dataset,
    out_folder,
    remi_tokenizer,
    max_decoder_len=1024,
    num_workers=8
):

    os.makedirs(out_folder, exist_ok=True)

    # Prepare args for each HF row
    args_list = [
        (item, out_folder, remi_tokenizer, max_decoder_len)
        for item in hf_dataset
    ]

    with Pool(num_workers) as p:
        list(
            tqdm(
                p.imap_unordered(process_item, args_list, chunksize=8),
                total=len(args_list)
            )
        )

    print("Pretokenization complete.")


In [ ]:
# ds_to_pretok = ds
ds_to_pretok = sample

pretokenize_gigamidi(
    hf_dataset=ds_to_pretok,
    out_folder="gigamidi_pretokenized",
    remi_tokenizer=REMI_TOKENIZER,
    max_decoder_len=1024,
    num_workers=8
)

  0%|          | 0/10 [00:00<?, ?it/s]

# MIDICaps

## Download MIDICaps MIDI Files

In [ ]:
from huggingface_hub import hf_hub_download
import tarfile
import os

# repo_id = "amaai-lab/MidiCaps"
# filename = "midicaps.tar.gz"

# # Download the file (cached locally)
# path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")

# # Optionally extract it
# out_dir = "./midi_caps_data"
# os.makedirs(out_dir, exist_ok=True)
# with tarfile.open(path, "r:gz") as tar:
#     tar.extractall(path=out_dir)

# print("Extracted to:", out_dir)

c:\Users\josep\miniforge3\envs\csc413\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\josep\.cache\huggingface\hub\datasets--amaai-lab--MidiCaps. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Extracted to: ./midi_caps_data


## Pretokenize MIDICaps Files

In [3]:
from datasets import load_dataset

midicaps = load_dataset("amaai-lab/MidiCaps", split="train")
midicaps

Dataset({
    features: ['location', 'caption', 'genre', 'genre_prob', 'mood', 'mood_prob', 'key', 'time_signature', 'tempo', 'tempo_word', 'duration', 'duration_word', 'chord_summary', 'chord_summary_occurence', 'instrument_summary', 'instrument_numbers_sorted', 'all_chords', 'all_chords_timestamps', 'test_set'],
    num_rows: 168385
})

### Sample Dataset (Optional)

In [4]:
import random

num_samples = 100

# Generate random indices
random_indices = random.sample(range(len(midicaps)), num_samples)

# Select the random samples
sample = midicaps.select(random_indices)

print(sample)

Dataset({
    features: ['location', 'caption', 'genre', 'genre_prob', 'mood', 'mood_prob', 'key', 'time_signature', 'tempo', 'tempo_word', 'duration', 'duration_word', 'chord_summary', 'chord_summary_occurence', 'instrument_summary', 'instrument_numbers_sorted', 'all_chords', 'all_chords_timestamps', 'test_set'],
    num_rows: 100
})


### Pretokenization

In [5]:
import os
import torch
from tqdm import tqdm
from pathlib import Path
from transformers import T5Tokenizer
from multiprocessing import Pool

def process_item(args):
    (
        item,
        midi_folder,
        out_folder,
        remi_tokenizer,
        t5_tokenizer,
        max_decoder_len,
        max_text_len
    ) = args

    caption = item["caption"]
    midi_loc = item["location"]

    midi_path = os.path.join(midi_folder, midi_loc)

    # Output path (same name but .pt)
    base = os.path.splitext(midi_loc)[0]
    out_path = os.path.join(out_folder, base + ".pt")

    # Skip if already pretokenized
    if os.path.exists(out_path):
        return None

    # Create subdirectories (mirrored structure)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # ---- MIDI → REMI ----
    tokens = remi_tokenizer(midi_path)
    if len(tokens.ids) == 0:
        midi = [
            remi_tokenizer["BOS_None"],
            remi_tokenizer["EOS_None"],
        ]
    else:
        midi = (
            [remi_tokenizer["BOS_None"]]
            + tokens.ids
            + [remi_tokenizer["EOS_None"]]
        )

    # Pad/truncate
    if len(midi) < max_decoder_len:
        labels = torch.nn.functional.pad(
            torch.tensor(midi),
            (0, max_decoder_len - len(midi))
        )
    else:
        labels = torch.tensor(midi[:max_decoder_len])

    labels = labels.long()

    # ---- Caption → T5 ----
    enc = t5_tokenizer(
        caption,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_text_len,
    )

    input_ids = enc["input_ids"][0]
    attention_mask = enc["attention_mask"][0]

    # ---- Save ----
    torch.save(
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        },
        out_path
    )

    return None


def pretokenize_midicaps(
    hf_items,
    midi_folder,
    out_folder,
    remi_tokenizer,
    max_decoder_len=1024,
    max_text_len=256,
    num_workers=8
):
    os.makedirs(out_folder, exist_ok=True)

    t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

    # Prepare arguments for each process
    args_list = [
        (
            item,
            midi_folder,
            out_folder,
            remi_tokenizer,
            t5_tokenizer,
            max_decoder_len,
            max_text_len
        )
        for item in hf_items
    ]

    with Pool(num_workers) as p:
        list(
            tqdm(
                p.imap_unordered(process_item, args_list, chunksize=8),
                total=len(args_list)
            )
        )

    print("Pretokenization complete.")

In [9]:
%view sentencepiece

UsageError: Line magic function `%view` not found.


In [6]:
ds_to_pretok = midicaps
# ds_to_pretok = sample

pretokenize_midicaps(
    hf_items=ds_to_pretok,
    midi_folder="midicaps",
    out_folder="midicaps_pretokenized",
    remi_tokenizer=REMI_TOKENIZER,
    max_decoder_len=1024,
    max_text_len=256,
    num_workers=8
)

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
